In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import plotly.subplots as sp
from sympy import symbols, solve
import matplotlib.pyplot as plt

In [2]:
def portfolio_performance(weights, mean_returns, cov_matrix, esg_np):
    returns = np.sum(mean_returns*weights)
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    esg = np.dot(esg_np.T, weights)
    return std, returns, esg

In [3]:
def random_port(num_port, mean_returns, cov_matrix, rf, esg_np):
    results = np.zeros((1, num_port))
    weights_record = []
    for i in range(num_port):
        weights = np.random.random(10)
        weights /= np.sum(weights)
        weights_record.append(weights)
        port_std, port_return, port_esg = portfolio_performance(weights, mean_returns, cov_matrix, esg_np)
        results[0, i] = port_std
        results[1, i] = port_return
        results[2, i] = (port_return - rf) / port_std
        results[3, i] = port_esg
    return results, weights_record

In [5]:
def neg_SR(weights, mean_return, cov_matrix, rf, esg_np):
    p_var, p_ret, p_esg = portfolio_performance(weights, mean_return, cov_matrix, esg_np)
    return -(p_ret - rf) / p_var

In [ ]:
def max_sharpe_ratio(weights, mean_return, cov_matrix, rf, esg_np):
    num_assets = len(mean_return)
    args = (mean_return, cov_matrix, rf, esg_np)
    constraints = ({'type' : 'eq', 'fun' : lambda x: np.sum(x) - 1})
    bound = (0.0, 1.0)
    bounds = tuple(bound) for asset in range(num_assets)
    result = sco.minimize(neg_SR, num_assets*[1./num_assets], args=args, method='SLSQP', bounds=bounds, constraints=constraints])

    return result
    

In [6]:
def neg_esg_ratio(weights, mean_return, cov_matrix, rf, esg_np):
    p_var, p_ret, p_esg = portfolio_performance(weights, mean_return, cov_matrix, esg_np)
    return - (((p_ret - rf) / p_var) + 2*p_esg)

In [ ]:
def max_esg_ratio(weights, mean_return, cov_matrix, rf, esg_np):
    num_assets = len(mean_return)
    args = (mean_return, cov_matrix, rf, esg_np)
    constraints = ({'type' : 'eq', 'fun' : lambda x: np.sum(x) - 1})
    bound = (0.0, 1.0)
    bounds = tuple(bound) for asset in range(num_assets)
    result = sco.minimize(neg_esg_ratio, num_assets*[1./num_assets], args=args, method='SLSQP', bounds=bounds, constraints=constraints])

    return result

In [7]:
def port_vol(weights, mean_return, cov_matrix, esg_np):
    portfolio_performance(weights, mean_return, cov_matrix, esg_np)[0]


In [ ]:
def min_variance(mean_return, cov_matrix, esg_np):
    num_assets = len(mean_return)
    args = (mean_return, cov_matrix, esg_np)
    constraints = ({'type' : 'eq', 'fun' : lambda x: np.sum(x) - 1})
    bound = (0.0, 1.0)
    bounds = tuple(bound) for asset in range(num_assets)
    result = sco.minimize(port_vol, num_assets*[1./num_assets], args=args, method='SLSQP', bounds=bounds, constraints=constraints])

    return result

In [ ]:
def efficient_return(mean_return, cov_matrix, target, esg_np):
    num_assets = len(mean_return)
    args = ((mean_return, cov_matrix, esg_np))
    
    def portfolio_return(weights):
        return portfolio_performance(weights, mean_return, cov_matrix, esg_np)
    

    constraints = ({'type' : 'eq', 'fun' : lambda x: np.sum(x) - target})
    bounds = tuple(0,1) for asset in range(num_assets)
    result = sco.minimize(port_vol, num_assets*[1./num_assets], args=args, method='SLSQP', bounds=bounds, constraints=constraints])

    return result

def efficient_frontier(mean_return, cov_matrix, returns_range, esg_np):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_return, cov_matrix, target, esg_np))
    return efficients
    

In [11]:
def display_calculated_ef_with_random(mean_return, cov_matrix, num_port, rf, esg_np):
    results, _ = random_port(num_port, mean_return, cov_matrix, rf, esg_np)

    max_sharpe = max_sharpe_ratio(mean_return, cov_matrix, rf, esg_np) #Dette er en optimizer funktion
    sdp, rp, esg = portfolio_performance(max_sharpe['x'], mean_return, cov_matrix, esg_np)
    max_sr = (rp-rf)/sdp
    max_SR_ESG = (((rp - rf) / sdp) + 2*esg)

    max_esg = max_esg_ratio(mean_return, cov_matrix, rf, esg_np) # OPtimizer
    esg_sdp, esg_rp, m_esg = portfolio_performance(max_esg['x'], mean_return, cov_matrix, esg_np)
    max_SR_esg = (esg_rp-rf)/esg_sdp
    max_ESG = (((esg_rp-rf)/esg_sdp) - 2*m_esg)

    min_vol = min_variance(mean_return, cov_matrix, esg_np)
    sdp_min, rp_min, esg_min = portfolio_performance(min_vol['x'], mean_return, cov_matrix, esg_np)
    min_SR = (rp_min - rf)/sdp_min
    min_ESG = (((rp_min - rf) / sdp_min) + 2*esg_min
    ####-------
    ####-----


   # "-"*80
   # Tangency portfolio
   # Annualised return: round(rp, 2)
   # VoL : round(sdp, 2)
   # Sharpe ratio: round(max_SR, 2)
   # ESG-SR Score: np.around(max_SR_ESG, 2)
   # Max_sharpe_allocation

   #"-"*80
   #Maximum ESG-SR portfolo
   # Return round(esg_rp, 2)
   # Vol: round_esg_sdp, 2)
   # Sharpe Ration, round(max_SR_esg, 2)
   #ESG SR score. np.around(max_ESG, 2)

   ##SÅ PRINTER DE OGSÅ FOR 
    plt.figure(figsize=(15,10))
    plt.scatter(results[0,:], results[1,:], c=results[2,:], cmap='V1GnEu', marker='0', s=10, alpha=0.3)
    plt.colorbar(label='Sharpe Ratio')
    plt.scatter(sdp,rp,marker='*', color='r', label='Tangency portfolio')
    plt.scatter(sdp_min, rp_min, marker='*', color='g', label='MVP')
    plt.scatter(esg_sdp, esg_rp, marker='*', color='b', label='MAX ESG-SR portfolio')
    target = np.linspace(rp_min, 0.22, 50)
    target2 = np.linspace(rp_min, 0.21, 50)
    efficient_portfolios = efficient_frontier(mean_return, cov_matrix, target, esg_np)
    plt.plot([p, 'fur']) for p in efficient_portfolios, target, linestyle = '-.', color='black', label='EF'
    plt.plot([p, 'fur']) for p in efficient_portfolios, target2, linestyle = '-.', color='black', label=' ESG EF'
    plt.xlabel('Vol')
    plt.ylabel('Return')
    plt.legend(labelspacing=0.8, fontsize=12)
